In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
from ipywidgets import interact
from scipy.stats import skew, kurtosis, mode
from scipy.spatial import KDTree
from scipy.ndimage import convolve
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans


In [2]:
file_path = r"\\192.168.1.100\CoreScan3-2\Acquisitions\Trailer-Calibration-Scans-DO-NOT-MOVE-OR-DELETE\AI21\Macassa3.5\AI21-Dec27-2024\Core_000_Box_001_of_999_Part_1_of_6"

In [3]:
coords_file = os.path.join(file_path, '.component_parameters.txt')
with open(coords_file) as file:
        lines = file.readlines()
            
        for line in lines:
            if "XRAY_DPP[Acquisition]#0.Y.Start:" in line:
                y_offset = (float)(line.split("XRAY_DPP[Acquisition]#0.Y.Start:")[1].strip())
            elif "XRAY_DPP[Acquisition]#0.X.Start:" in line:
                x_start = (float)(line.split("XRAY_DPP[Acquisition]#0.X.Start:")[1].strip())
            elif "XRAY_DPP[Acquisition]#0.X.Stop:" in line:
                x_stop = (float)(line.split("XRAY_DPP[Acquisition]#0.X.Stop:")[1].strip())

In [6]:
point_cloud = []

if os.path.isdir(file_path):
    lidar_files = [fn for fn in os.listdir(
        file_path) if fn.endswith('.bpc')]
    if any(lidar_files):
        lidar_filename = file_path + os.sep + lidar_files[0]
lidar_filename = r'\\192.168.1.100\CoreScan3-2\Acquisitions\Trailer-Calibration-Scans-DO-NOT-MOVE-OR-DELETE\AI21\Macassa3.5\AI21-Dec27-2024\Core_000_Box_001_of_999_Part_5_of_6'
data = np.fromfile(lidar_filename, dtype=np.float32)
point_cloud = data.reshape(-1, 3)  # to xyz

ff = ~np.isnan(point_cloud).any(axis=1)
point_cloud = point_cloud[ff, ...]

point_cloud[:, 1] = point_cloud[:, 1] - float(y_offset)

print(f"{file_path} is loaded. \n# of point {point_cloud.shape[0]}")

PermissionError: [Errno 13] Permission denied: '\\\\192.168.1.100\\CoreScan3-2\\Acquisitions\\Trailer-Calibration-Scans-DO-NOT-MOVE-OR-DELETE\\AI21\\Macassa3.5\\AI21-Dec27-2024\\Core_000_Box_001_of_999_Part_5_of_6'

In [ ]:

matrix_file = (os.path.join(file_path, ".XRAY_DPP_001.lidar2xrf"))
with open(matrix_file, 'r') as file:
    lines = file.readlines()

transformation_matrix = np.array([list(map(float, line.strip().split(","))) for line in lines])

num_points = point_cloud.shape[0]

homogeneous_points = np.hstack((point_cloud, np.ones((num_points, 1))))
transformed_points = homogeneous_points @ transformation_matrix.T
point_cloud = transformed_points[:, :3]




print(f"# of point {point_cloud.shape[0]}")

In [ ]:
def trim_cloud(data):
    floor = mode(data[:, 2])[0] - 10
    print(floor)
    data[:,2] = floor - data[:, 2] 
    data = data[data[:, 2] > 0]
    data = data[
    (data[:,0] >= x_stop) & 
    (data[:,0] <= x_start) 
    ]
    return data

point_cloud = trim_cloud(point_cloud)

def point_matrix(data = point_cloud):
    point_matrix = [[]]
    matrix_x = np.unique(data[:,0])
    matrix_y = np.unique(data[:,1])
    
    for x in matrix_x.length():
        for y in matrix_y.length():
            point_matrix[x][y] = data[
                data[:,0] == matrix_x(x) & 
                (data[:,1] == matrix_y(y))][:,2]
            
    return point_matrix
            
#matrix = point_matrix()

def gaussian_kernel(size: int, sigma: float):
    kernel = np.linspace(-size // 2, size // 2, size)
    kernel = np.exp(-0.5 * (kernel / sigma) ** 2)
    kernel /= np.sum(kernel) 
    return kernel


def apply_gaussian(kernel_size=10, sigma=1.0):
    y_vals = np.unique(point_cloud[:,1])
    kernel = gaussian_kernel(kernel_size, sigma)
    smoothed_point_cloud = point_cloud.copy()
    print(kernel)
    for y in y_vals:
        indices = np.where(point_cloud[:,1] == y)[0]
        
        z_values = point_cloud[indices, 2]
        
        smoothed_z = convolve(z_values, kernel, mode='nearest')  
        
        smoothed_point_cloud[indices, 2] = smoothed_z
        
    return smoothed_point_cloud

def convolved_data(data,kernel):
    x_values = np.unique(data[:,0])
    y_values = np.unique(data[:,1])
    array = [[None for _ in range(len(y_values))] for _ in range(len(x_values))]
    map = {(row[0],row[1]): (index, row[2]) for index,row in enumerate(data)}

    for x in range (len(x_values)):
        for y in range (len(y_values)):
            x_val = x_values[x]
            y_val = y_values[y]
            z = map.get((x_val,y_val))
            if z is not None:
                array[x][y] = z[1]
            else:
                array[x][y] = 0
    
    array = convolve(array,kernel, mode = 'reflect')

    data2 = data.copy()

    for x in range (len(x_values)):
        for y in range (len(y_values)):
            x_val = x_values[x]
            y_val = y_values[y]
            index = map.get((x_val,y_val))
            if index is not None:
                data2[index[0],2] = array[x][y]
    
    return data2

#point_cloud = apply_gaussian(30,5)

print(f"# of point {point_cloud.shape[0]}")

point_tree = KDTree(point_cloud[:,:2])

point_cloud_copy = point_cloud.copy()











kernel = np.array([
    [1, -2,  1],
    [1, -2,  1],
    [1, -2,  1]
])

point_cloud = convolved_data(point_cloud_copy, kernel)

x = point_cloud[:, 0]
y = point_cloud[:, 1]
z = point_cloud[:, 2]

downsampled_indices = np.random.choice(point_cloud.shape[0], size=500000, replace=False)
x_downsampled = x[downsampled_indices]
y_downsampled = y[downsampled_indices]
z_downsampled = z[downsampled_indices]

fig = plt.figure(figsize=(25, 10))

ax_scatter = fig.add_subplot(1, 2, 2)
scatter = ax_scatter.scatter(
    x_downsampled,
    y_downsampled,
    c=z_downsampled,
    cmap='binary',  
    s = 8
)

plt.show()

x = point_cloud_copy[:, 0]
y = point_cloud_copy[:, 1]
z = point_cloud_copy[:, 2]

downsampled_indices = np.random.choice(point_cloud_copy.shape[0], size=500000, replace=False)
x_downsampled = x[downsampled_indices]
y_downsampled = y[downsampled_indices]
z_downsampled = z[downsampled_indices]

fig = plt.figure(figsize=(25, 10))

ax_scatter = fig.add_subplot(1, 2, 2)
scatter = ax_scatter.scatter(
    x_downsampled,
    y_downsampled,
    c=z_downsampled,
    cmap='binary',  
    s = 8
)

plt.show()


In [ ]:
def get_core_type(properties):
    mean = 45
    if (mean > properties[0]):
        if(properties[1] < 1.5):
            return "half"
        if (properties[3] > -1):
            return "empty"
        else:
            return "rubble/marker"
    else:
        return "full"


def core_types(x_span = 1, y_span = 12,y_index = y_offset):
    types = {}
    properties = core_props(x_span, y_span, y_index)
    for x in properties.keys():
        types[x] = get_core_type(properties[x])
    return types

def core_props(x_span = 1, y_span = 12,y_index = y_offset, step = 1):
    x_values = np.unique(point_cloud[:, 0])
    core_types = {}
    y = y_index
    print(y_offset)
    x = x_values.min()
    while(x < x_values.max()):
        distribution = get_distribution([x, y], x_span, y_span) 
        core_types[x] = get_props(distribution)
        x += step
    return core_types

def get_vectors(x_span = 1, y_span = 12, step = 1,y_index = y_offset,):
    vectors = {}
    x_values = np.unique(point_cloud[:, 0])
    y = y_index
    print(y_offset)
    x = x_values.min()
    while(x < x_values.max()):
        distribution = get_distribution([x, y], x_span, y_span) 
        properties = get_props(distribution)
        if not np.any(np.isnan(properties)):
            vectors[x] = properties
        x += step
    return vectors


def get_distribution(point=[0, 0], x_span=10, y_span=15):
    search_radius = max(x_span, y_span)
    
    indices = point_tree.query_ball_point(point, search_radius)
    
    result_points = point_cloud[indices]

    filtered_points = result_points[
        (result_points[:, 0] >= point[0] - x_span) & (result_points[:, 0] <= point[0] + x_span) &
        (result_points[:, 1] >= point[1] - y_span) & (result_points[:, 1] <= point[1] + y_span)
    ]
    
    return filtered_points[:, 2]

def get_props(distribution):
    properties = []

    mean = np.mean(distribution)
    variance =  np.var(distribution)
    std = np.std(distribution)
    skw = skew(distribution)
    kurt = kurtosis(distribution)

    properties.append(mean)
    properties.append(variance)
    properties.append(std)
    properties.append(skw)
    properties.append(kurt)
    
    return properties

def get_eigenvalues(point=[0, 0], x_span=10, y_span=15):
    search_radius = max(x_span, y_span)
    
    indices = point_tree.query_ball_point(point, search_radius)
    
    result_points = point_cloud[indices]

    filtered_points = result_points[
        (result_points[:, 0] >= point[0] - x_span) & (result_points[:, 0] <= point[0] + x_span) &
        (result_points[:, 1] >= point[1] - y_span) & (result_points[:, 1] <= point[1] + y_span)
    ]

    cov_matrix = np.cov(filtered_points.T)
    return np.linalg.eigvals(cov_matrix)




def get_props_string(distribution):
    mean = np.mean(distribution)
    variance =  np.var(distribution)
    std = np.std(distribution)
    skw = skew(distribution)
    kurt = kurtosis(distribution)
        
    properties_text = (
    f"Mean: {mean:.4f}\n"
    f"Variance: {variance:.4f}\n"
    f"Standard Deviation: {std:.4f}\n"
    f"Skewness: {skw:.4f}\n"
    f"Kurtosis: {kurt:.4f}")

    return properties_text

In [ ]:
x = point_cloud[:, 0]
y = point_cloud[:, 1]
z = point_cloud[:, 2]

downsampled_indices = np.random.choice(point_cloud.shape[0], size=500000, replace=False)
x_downsampled = x[downsampled_indices]
y_downsampled = y[downsampled_indices]
z_downsampled = z[downsampled_indices]

def view_histogram(x_index, y_index, x_span, y_span):
    fig = plt.figure(figsize=(25, 10))

    ax_scatter = fig.add_subplot(1, 2, 2)
    scatter = ax_scatter.scatter(
        x_downsampled,
        y_downsampled,
        c=z_downsampled,
        cmap='binary',  
        s = 8
    )

    ax_scatter.hlines(y = y_index + y_span, xmin = (x_index - x_span), xmax = (x_index + x_span), color='red', linestyle='solid', linewidth=1)
    ax_scatter.hlines(y = y_index - y_span, xmin = (x_index - x_span), xmax = (x_index + x_span), color='red', linestyle='solid', linewidth=1)
    ax_scatter.vlines(x = x_index + x_span, ymin = (y_index - y_span), ymax = (y_index + y_span), color='red', linestyle='solid', linewidth=1)
    ax_scatter.vlines(x = x_index - x_span, ymin = (y_index - y_span), ymax = (y_index + y_span), color='red', linestyle='solid', linewidth=1)
    ax_scatter.axhline(y = y_index, color='red', linestyle='--', linewidth=2, label = 'core number {y_index}')

    ax_scatter.set_title('LIDAR data heatmap')
    ax_scatter.set_xlabel('X-axis')
    ax_scatter.set_ylabel('Y-axis')
    ax_scatter.legend()
    plt.colorbar(scatter, ax=ax_scatter, label='Z-axis Values')

    histogram = fig.add_subplot(2, 2, 1)

    distribution = get_distribution([x_index, y_index], x_span, y_span)

    histogram.hist(distribution, bins = 500)

    histogram.set_title(f'Z  value distribution for window at X = {x_index}') 
    histogram.set_xlabel('Z value')
    histogram.set_ylabel('count')

    plt.gcf().text(
    0.25, 0.2, (f"{get_props_string(distribution)} \n {get_core_type(get_props(distribution))}"), # \n {get_eigenvalues([x_index, y_index], x_span, y_span)}"), 
    fontsize=30, ha='center', va='bottom', bbox=dict(boxstyle="round,pad=0.3", edgecolor='black', facecolor='white')
)

    plt.tight_layout()
    plt.show()

interact(
    view_histogram,
    x_index=(int(x_downsampled.min()), int(x_downsampled.max()), 1),
    y_index=(int(y_downsampled.min()), int(y_downsampled.max()), 1),
    x_span = (0, 50, 1),
    y_span = (1, 20, 1)
)

In [ ]:
core_types_dict = core_types(x_span=5, y_span=10)

x_values = list(core_types_dict.keys())
core_type_labels = list(core_types_dict.values())

unique_labels = list(set(core_type_labels))
colors = {label: f"C{i}" for i, label in enumerate(unique_labels)} 

plt.figure(figsize=(20, 2))
for x, core_type in zip(x_values, core_type_labels):
    plt.bar(x, height=1, width = 1, color=colors[core_type], edgecolor="none", label=core_type if core_type not in plt.gca().get_legend_handles_labels()[1] else None)

plt.title("Core Type by X-Value")
plt.xlabel("X-Value")
plt.ylabel("Core Type")
plt.legend(title="Core Type", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()





In [ ]:

def view_values(y_index, x_span, y_span, step):
    fig = plt.figure(figsize=(25, 10))
    gs = fig.add_gridspec(5, 2, width_ratios=[1, 3], height_ratios=[1, 1, 1, 1, 1], wspace=0.3)


    ax_scatter = fig.add_subplot(gs[:, 1])
    scatter = ax_scatter.scatter(
        x_downsampled,
        y_downsampled,
        c=z_downsampled,
        cmap='binary',  
        s = 8
    )

    ax_scatter.axhline(y = y_index + y_span, color='red', linestyle='--', linewidth=1)
    ax_scatter.axhline(y = y_index - y_span, color='red', linestyle='--', linewidth=1)

    ax_scatter.set_title('LIDAR data heatmap')
    ax_scatter.set_xlabel('X-axis')
    ax_scatter.set_ylabel('Y-axis')
    plt.colorbar(scatter, ax=ax_scatter, label='Z-axis Values')

    
    properties = core_props(x_span, y_span, y_index, step)

    plots = []

    for i in range(5):
        plots.append(fig.add_subplot(gs[i, 0]))
        plots[i].plot(properties.keys(), [prop[i] for prop in properties.values()])
        plots[i].set_ylabel(f'{["Mean", "Variance", "Standard Deviation", "Skewness", "Kurtosis"][i]}')
        plots[i].set_xlabel('X-axis')
       
    plt.show()

interact(
    view_values,
    y_index=(int(y_downsampled.min()), int(y_downsampled.max()), 1),
    x_span = (0, 50, 1),
    y_span = (1, 20, 1),
    step = (1, 10, 1)
)

In [ ]:
def view_values(y_index, x_span, y_span, step):
    fig = plt.figure(figsize=(20, 12))
    gs = fig.add_gridspec(5, 5, width_ratios=[1,1,1,1,1], height_ratios=[1, 1, 1, 1, 1], wspace=0.3, hspace=0.3)


    ax_scatter = fig.add_subplot(gs[:2,3:6])
    scatter = ax_scatter.scatter(
        x_downsampled,
        y_downsampled,
        c=z_downsampled,
        cmap='binary',  
        s = 8
    )

    ax_scatter.axhline(y = y_index + y_span, color='red', linestyle='--', linewidth=1)
    ax_scatter.axhline(y = y_index - y_span, color='red', linestyle='--', linewidth=1)

    ax_scatter.set_title('LIDAR data heatmap')
    ax_scatter.set_xlabel('X-axis')
    ax_scatter.set_ylabel('Y-axis')
    plt.colorbar(scatter, ax=ax_scatter, label='Z-axis Values')

    
    properties = core_props(x_span, y_span, y_index, step)

    plots = []
    names = ["Mean", "Variance", "Standard Deviation", "Skewness", "Kurtosis"]
    index = 0
    for i in range(5):
        for j in range(i + 1):
            plots.append(fig.add_subplot(gs[i, j]))
            plots[index].scatter([prop[j] for prop in properties.values()], [prop[i] for prop in properties.values()])
            plots[index].set_ylabel(f'{names[i]}')
            plots[index].set_xlabel(f'{names[j]}')
            index += 1
       
    plt.show()

interact(
    view_values,
    y_index=(int(y_downsampled.min()), int(y_downsampled.max()), 1),
    x_span = (0, 50, 1),
    y_span = (1, 20, 1),
    step = (1, 10, 1)
)

In [ ]:
x_span = 5
y_span = 15
step = 5

v_dict = get_vectors(x_span,y_span,step)
print(v_dict.values())

kmeans = KMeans(n_clusters = 4)
kmeans.fit(list(v_dict.values()))


labels = kmeans.labels_
x_values = list(v_dict.keys())


plt.figure(figsize=(20, 2))
for x, core_type in zip(x_values, labels):
    plt.bar(x, height=1, width = step, color=f"C{core_type**2}", edgecolor="none")

plt.title("Core Type by X-Value")
plt.xlabel("X-Value")
plt.yticks([])
plt.show()
